### NHANES Health Data Analysis with Bokeh 
* Name:LI WAN
* Student Number:223718804
* E-mail:s223718804@deakin.edu.au
* Course:SIT731

---

### Introduction 
The exploration of potential correlations within the NHANES Program 2017-2020 Pre-Pandemic March datasets provides valuable insights into the relationships between demographic variables, physical health indicators, cardiovascular conditions, dietary habits, and occupational factors. In this report, we focus on five key datasets: Demographic Variables and Sample Weights (P_DEMO), Body Measures (P_BMX), Blood Pressure & Cholesterol (P_BPQ), Dietary Supplement Use 30-Day - Total Dietary Supplements (P_DSQTOT), and Oral Health - Recommendation of Care (P_OHXREF). Through data wrangling and visualization using Bokeh packages, we aim to uncover patterns and associations that contribute to a deeper understanding of health-related aspects in the sampled population.

### Data Selection
The NHANES (National Health and Nutrition Examination Survey) Program 2017-2020 Pre-Pandemic March provides six categories, Demogra Laboratory Dataaphics Data, Dietary Data, Examination Data, Questionnaire Data, Limited Access Data. Limited Access Data of which Limited Access Data we could not access for this study.  
This report wants to explore whether there is a potential correlation between population, physical data, cardiovascular disease, food, and occupation. Therefore, we will choose 5 dataset. Demographic Variables and Sample Weights(P_DEMO), Body Measures (P_BMX), Blood Pressure & Cholesterol (P_BPQ), Dietary Supplement Use 30-Day - Total Dietary Supplements (P_DSQTOT) and Oral Health - Recommendation of Care (P_OHXREF). Next, we will fetch each dataset and perform data wrangling specifically based on the data.

### Data Wrangling
1. Demographic Variables and Sample Weights  
Firstly, we're going to work with demographic data to see whether there are NaN, missing data.  
Data Source: https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DEMO.htm  
Codebook of P_DEMO

* SEQN - Respondent sequence number
* SDDSRVYR - Data release cycle
* RIDSTATR - Interview/Examination status
* RIAGENDR - Gender
* RIDAGEYR - Age in years at screening
* RIDAGEMN - Age in months at screening - 0 to 24 mos
* RIDRETH1 - Race/Hispanic origin
* RIDRETH3 - Race/Hispanic origin w/ NH Asian
* RIDEXMON - Six-month time period
* DMDBORN4 - Country of birth
* DMDYRUSZ - Length of time in US
* DMDEDUC2 - Education level - Adults 20+
* DMDMARTZ - Marital status
* RIDEXPRG - Pregnancy status at exam
* SIALANG - Language of SP Interview
* SIAPROXY - Proxy used in SP Interview?
* SIAINTRP - Interpreter used in SP Interview?
* FIALANG - Language of Family Interview
* FIAPROXY - Proxy used in Family Interview?
* FIAINTRP - Interpreter used in Family Interview?
* MIALANG - Language of MEC Interview
* MIAPROXY - Proxy used in MEC Interview?
* MIAINTRP - Interpreter used in MEC Interview?
* AIALANGA - Language of ACASI Interview
* WTINTPRP - Full sample interview weight
* WTMECPRP - Full sample MEC exam weight
* SDMVPSU - Masked variance pseudo-PSU
* SDMVSTRA - Masked variance pseudo-stratum
* INDFMPIR - Ratio of family income to poverty

In [112]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from bokeh.io import output_notebook
from bokeh.models import CustomJSTickFormatter, Label
from bokeh.palettes import DarkText, Vibrant3 as colors
from bokeh.plotting import figure, show 

In [113]:
# define file URL
xpt_file_url = 'https://github.com/Maxmelon326/SIT731/raw/main/P_DEMO.XPT'

# read XPT file
demo= pd.read_sas(xpt_file_url, format='xport')

# print shape and head
print(demo.shape)
demo.head()

(15560, 29)


,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,DMDBORN4,...,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANGA,WTINTPRP,WTMECPRP,SDMVPSU,SDMVSTRA,INDFMPIR
0,109263.0,66.0,2.0,1.0,2.0,NaN,5.0,6.0,2.0,1.0,...,2.0,NaN,NaN,NaN,NaN,7891.762435,8.951816e+03,3.0,156.0,4.66
1,109264.0,66.0,2.0,2.0,13.0,NaN,1.0,1.0,2.0,1.0,...,2.0,1.0,2.0,2.0,1.0,11689.747264,1.227116e+04,1.0,155.0,0.83
2,109265.0,66.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,1.0,...,2.0,NaN,NaN,NaN,NaN,16273.825939,1.665876e+04,1.0,157.0,3.06
3,109266.0,66.0,2.0,2.0,29.0,NaN,5.0,6.0,2.0,2.0,...,2.0,1.0,2.0,2.0,1.0,7825.646112,8.154968e+03,2.0,168.0,5.00
4,109267.0,66.0,1.0,2.0,21.0,NaN,2.0,2.0,NaN,2.0,...,2.0,NaN,NaN,NaN,NaN,26379.991724,5.397605e-79,1.0,156.0,5.00


We don't need all the data, in order to make the data look clearer, we pick out the ones we need according to the definition of the variables. Based on the code book of DEMO, we will choose:
* SEQN(Respondent sequence number);  
* RIAGENDR(Gender):  
1-Male,  
2-Female;  
* RIDAGEYR(Age in years at screening);  
* DMDBORN4 - Country of birth:  
1-born in 50 US states or Washington, DC,  
2-Others,  
3-Refused,  
99-Don't Know;
* Ratio of family income to poverty(INDFMPIR):  
0 to 4.98:Range of Values,  
5:Value greater than or equal to 5.00.  
Since SEQN (Respondent sequence number) is a foreign key to connect other dataset data, it cannot be concatenated if when SEQN=NULL, so delete these null data.

In [114]:
selected_columns = ['SEQN', 'RIAGENDR', 'RIDAGEYR', 'DMDBORN4','INDFMPIR']

# select columns of demo and rename
demo_selected = demo[selected_columns].copy()
demo_selected.columns = ['Respondent sequence number', 'Gender', 'Age', 'Birth Country','Poverty']


#null_rows = demo_selected[demo_selected['Respondent sequence number'].isnull()]
#null_rows.head() now null in SEQN
# Replace values in 'Gender' column
demo_selected['Gender'].replace({1: 'Male', 2: 'Female'}, inplace=True)
demo_selected['Birth Country'].replace({1: 'USA', 2: 'Others',77: np.nan, 99:np.nan}, inplace=True)
# Because Refused can not provide information, Replace with NaN
# Because Don't know can not provide information, Replace with NaN

# DROP SEQN=null
demo_selected= demo_selected.dropna(subset=['Respondent sequence number','Gender', 'Age','Birth Country','Poverty'])

print(demo_selected.shape)
#Print
demo_selected.head()

(13357, 5)


,Respondent sequence number,Gender,Age,Birth Country,Poverty
0,109263.0,Male,2.0,USA,4.66
1,109264.0,Female,13.0,USA,0.83
2,109265.0,Male,2.0,USA,3.06
3,109266.0,Female,29.0,Others,5.00
4,109267.0,Female,21.0,Others,5.00


2. Body Measures  
Secondly, we're going to work with body measures to see whether there are NaN, missing data.  
Data Source:https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_BMX.htm  
Codebook of P_BMX
* SEQN - Respondent sequence number
* BMDSTATS - Body Measures Component Status Code
* BMXWT - Weight (kg)
* BMIWT - Weight Comment
* BMXRECUM - Recumbent Length (cm)
* BMIRECUM - Recumbent Length Comment
* BMXHEAD - Head Circumference (cm)
* BMIHEAD - Head Circumference Comment
* BMXHT - Standing Height (cm)
* BMIHT - Standing Height Comment
* BMXBMI - Body Mass Index (kg/m**2)
* BMDBMIC - BMI Category - Children/Youth
* BMXLEG - Upper Leg Length (cm)
* BMILEG - Upper Leg Length Comment
* BMXARML - Upper Arm Length (cm)
* BMIARML - Upper Arm Length Comment
* BMXARMC - Arm Circumference (cm)
* BMIARMC - Arm Circumference Comment
* BMXWAIST - Waist Circumference (cm)
* BMIWAIST - Waist Circumference Comment
* BMXHIP - Hip Circumference (cm)
* BMIHIP - Hip Circumference Comment

In [115]:
# define file URL
xpt_file_url = 'https://github.com/Maxmelon326/SIT731/raw/main/P_BMX.XPT'

# read XPT file
bmx= pd.read_sas(xpt_file_url, format='xport')

# print shape and head
print(bmx.shape)
bmx.head()

(14300, 22)


,SEQN,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,...,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXHIP,BMIHIP
0,109263.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109264.0,1.0,42.2,NaN,NaN,NaN,NaN,NaN,154.7,NaN,...,36.3,NaN,33.8,NaN,22.7,NaN,63.8,NaN,85.0,NaN
2,109265.0,1.0,12.0,NaN,91.6,NaN,NaN,NaN,89.3,NaN,...,NaN,NaN,18.6,NaN,14.8,NaN,41.2,NaN,NaN,NaN
3,109266.0,1.0,97.1,NaN,NaN,NaN,NaN,NaN,160.2,NaN,...,40.8,NaN,34.7,NaN,35.8,NaN,117.9,NaN,126.1,NaN
4,109269.0,3.0,13.6,NaN,90.9,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN


We don't need all the data, in order to make the data look clearer, we pick out the ones we need according to the definition of the variables. Based on the code book of BMX, we will choose SEQN (Respondent sequence number), BMXWT(Weight (kg)), BMXHT(Standing Height (cm)), BMXBMI(Body Mass Index (kg/m**2)). For futher analysis ,we will delete null data.

In [116]:
selected_columns = ['SEQN', 'BMXWT', 'BMXHT','BMXWAIST', 'BMXBMI']

# select columns of demo and rename
bmx_selected = bmx[selected_columns].copy()
bmx_selected.columns = ['Respondent sequence number', 'Weight', 'Height','Waist', 'BMI']


#null_rows = bmx_selected[bmx_selected['Respondent sequence number'].isnull()| bmx_selected['BMI'].isnull()]
#null_rows.head() #now null in SEQN and but null in bmi

# DROP SEQN & BMI =null
bmx_selected = bmx_selected.dropna(subset=['Respondent sequence number','BMI'])

print(bmx_selected.shape)
#Print
bmx_selected.head()

(13137, 5)


,Respondent sequence number,Weight,Height,Waist,BMI
1,109264.0,42.2,154.7,63.8,17.6
2,109265.0,12.0,89.3,41.2,15.0
3,109266.0,97.1,160.2,117.9,37.8
5,109270.0,75.3,156.0,91.4,30.9
6,109271.0,98.8,182.3,120.4,29.7


3. Blood Pressure & Cholesterol (P_BPQ)  
Thirdly, we're going to work with Blood Pressure & Cholesterol data to see whether there are NaN, missing data.   
Data Source:https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_BPQ.htm  
Codebook of P_BPQ
* SEQN - Respondent sequence number
* BPQ020 - Ever told you had high blood pressure
* BPQ030 - Told had high blood pressure - 2+ times
* BPD035 - Age told had hypertension
* BPQ040A - Taking prescription for hypertension
* BPQ050A - Now taking prescribed medicine for HBP
* BPQ080 - Doctor told you - high cholesterol level
* BPQ060 - Ever had blood cholesterol checked
* BPQ070 - When blood cholesterol last checked
* BPQ090D - Told to take prescriptn for cholesterol
* BPQ100D - Now taking prescribed medicine

In [117]:
# define file URL
xpt_file_url = 'https://github.com/Maxmelon326/SIT731/raw/main/P_BPQ.XPT'

# read URL XPT file
bpq= pd.read_sas(xpt_file_url, format='xport')

# print shape and head
print(bpq.shape)
bpq.head()

(10195, 11)


,SEQN,BPQ020,BPQ030,BPD035,BPQ040A,BPQ050A,BPQ080,BPQ060,BPQ070,BPQ090D,BPQ100D
0,109266.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,NaN
1,109267.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,NaN
2,109268.0,2.0,NaN,NaN,NaN,NaN,2.0,9.0,NaN,NaN,NaN
3,109271.0,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0
4,109273.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,4.0,2.0,NaN


We don't need all the data, in order to make the data look clearer, we pick out the ones we need according to the definition of the variables. Based on the code book of BPQ, we will choose:  
*  SEQN (Respondent sequence number); 
* BPQ030(Told had high blood pressure - 2+ times):  
1-Yes,  
2-No,  
7-Refused,  
9-Don't know;  
* BPQ050A(Now taking prescribed medicine for HBP):  
1-Yes,  
2-No,  
7-Refused,  
9-Don't know; 
* BPQ100D(Now taking prescribed medicine for cholesterol):  
1-Yes,  
2-No,  
7-Refused,  
9-Don't know.   
For futher analysis ,we will delete null data.

In [118]:
selected_columns = ['SEQN', 'BPQ030', 'BPQ050A','BPQ100D']

# select columns of demo and rename
bpq_selected = bpq[selected_columns].copy()
bpq_selected.columns = ['Respondent sequence number', 'Told had hypertension', 'Taking medicine for hypertension','Taking medicine for cholesterol']

# Replace values in specified columns
replace_dict = {
    1: 'Yes',
    2: 'No',
    7: np.nan,  # Because Refused can not provide information, Replace with NaN
    9: np.nan,  # Because Don't know can not provide informationReplace with NaN
}
bpq_selected.replace(replace_dict, inplace=True)
# DROP null
bpq_selected = bpq_selected.dropna(subset=['Respondent sequence number', 'Told had hypertension', 'Taking medicine for hypertension','Taking medicine for cholesterol'])

print(bpq_selected.shape)
#Print
bpq_selected.head()

(1847, 4)


,Respondent sequence number,Told had hypertension,Taking medicine for hypertension,Taking medicine for cholesterol
5,109274.0,Yes,Yes,Yes
12,109290.0,No,Yes,Yes
27,109316.0,Yes,Yes,Yes
36,109330.0,Yes,Yes,Yes
53,109354.0,Yes,Yes,Yes


4. Dietary Supplement Use 30-Day - Total Dietary Supplements (P_DSQTOT)  
Fourthly, we're going to work with Blood Pressure & Cholesterol data to see whether there are NaN, missing data.   
Data Source:https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_DSQTOT.htm  
Codebook of P_DSQTOT
* SEQN - Respondent sequence number
* DSDCOUNT - Total # of Dietary Supplements Taken
* DSDANCNT - Total # of Antacids Taken
* DSD010 - Any Dietary Supplements Taken?
* DSD010AN - Any Antacids Taken?
* DSQTKCAL - Energy (kcal)
* DSQTPROT - Protein (gm)
* DSQTCARB - Carbohydrate (gm)
* DSQTSUGR - Total sugars (gm)
* DSQTFIBE - Dietary fiber (gm)
* DSQTTFAT - Total fat (gm)
* DSQTSFAT - Total saturated fatty acids (gm)
* DSQTMFAT - Total monounsaturated fatty acids (gm)
* DSQTPFAT - Total polyunsaturated fatty acids (gm)
* DSQTCHOL - Cholesterol (mg)
* DSQTLYCO - Lycopene (mcg)
* DSQTLZ - Lutein + zeaxanthin (mcg)
* DSQTVB1 - Thiamin (Vitamin B1) (mg)
* DSQTVB2 - Riboflavin (Vitamin B2) (mg)
* DSQTNIAC - Niacin (mg)
* DSQTVB6 - Vitamin B6 (mg)
* DSQTFA - Folic acid (mcg)
* DSQTFDFE - Folate, DFE (mcg)
* DSQTCHL - Total choline (mg)
* DSQTVB12 - Vitamin B12 (mcg)
* DSQTVC - Vitamin C (mg)
* DSQTVK - Vitamin K (mcg)
* DSQTVD - Vitamin D (D2 + D3) (mcg)
* DSQTCALC - Calcium (mg)
* DSQTPHOS - Phosphorus (mg)
* DSQTMAGN - Magnesium (mg)
* DSQTIRON - Iron (mg)
* DSQTZINC - Zinc (mg)
* DSQTCOPP - Copper (mg)
* DSQTSODI - Sodium (mg)
* DSQTPOTA - Potassium (mg)
* DSQTSELE - Selenium (mcg)
* DSQTCAFF - Caffeine (mg)
* DSQTIODI - Iodine (mcg)

In [119]:
# define file URL
xpt_file_url = 'https://github.com/Maxmelon326/SIT731/raw/main/P_DSQTOT.XPT'

# read URL XPT file
dsqtot= pd.read_sas(xpt_file_url, format='xport')

# print shape and head
print(dsqtot.shape)
dsqtot.head()

(15560, 39)


,SEQN,DSDCOUNT,DSDANCNT,DSD010,DSD010AN,DSQTKCAL,DSQTPROT,DSQTCARB,DSQTSUGR,DSQTFIBE,...,DSQTPHOS,DSQTMAGN,DSQTIRON,DSQTZINC,DSQTCOPP,DSQTSODI,DSQTPOTA,DSQTSELE,DSQTCAFF,DSQTIODI
0,109263.0,2.000000e+00,5.397605e-79,1.0,2.0,18.0,NaN,3.5,3.0,NaN,...,NaN,NaN,NaN,1.4,NaN,2.5,NaN,NaN,NaN,21.0
1,109264.0,5.397605e-79,5.397605e-79,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,109265.0,5.397605e-79,5.397605e-79,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109266.0,5.397605e-79,5.397605e-79,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,109267.0,1.000000e+00,5.397605e-79,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We don't need all the data, in order to make the data look clearer, we pick out the ones we need according to the definition of the variables. Based on the code book of DSQTOT, we will choose: 
* SEQN (Respondent sequence number);  
* DSQTSUGR - Total sugars (gm), 
* DSQTVC - Vitamin C (mg).  
For futher analysis ,we will delete null data.

In [120]:
selected_columns = ['SEQN','DSQTSUGR','DSQTVC']

# select columns of demo and rename
dsqtot_selected = dsqtot[selected_columns].copy()
dsqtot_selected.columns = ['Respondent sequence number', 'Total sugars','Vitamin C']


# DROP null
dsqtot_selected = dsqtot_selected.dropna(subset=['Respondent sequence number', 'Total sugars','Vitamin C'])

print(dsqtot_selected.shape)
#Print
dsqtot_selected.head()

(1633, 3)


,Respondent sequence number,Total sugars,Vitamin C
0,109263.0,3.00,13.0
12,109275.0,2.00,30.0
41,109304.0,1.07,21.3
47,109310.0,2.50,75.0
49,109312.0,3.00,530.0


5. Oral Health - Recommendation of Care (P_OHXREF)  
Fifthly, we're going to work with Oral health data to see whether there are NaN, missing data.   
Codebook of P_OHXREF  
Data source:https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/P_OHXREF.htm
* SEQN - Respondent sequence number
* OHDEXSTS - Overall Oral Health Exam Status
* OHDRCSTS - Recommendation Care Status Code
* OHAREC - Overall recommendation for care
* OHAROCDT - Decayed teeth
* OHAROCGP - Gum disease/problem
* OHAROCOH - Oral hygiene
* OHAROCCI - Impression of soft tissue condition
* OHAROCDE - Denture/Partial Denture/Plates
* OHARNF - No significant findings
* OHAROTH - Some other finding (specify w/referral)
* OHAPOS - Was participant exam position recumbent

In [121]:
# define file URL
xpt_file_url = 'https://github.com/Maxmelon326/SIT731/raw/main/P_OHXREF.XPT'

# read URL XPT file
ohxref= pd.read_sas(xpt_file_url, format='xport')

# print shape and head
print(ohxref.shape)
ohxref.head()

(13772, 12)


,SEQN,OHDEXSTS,OHDRCSTS,OHAREC,OHAROCDT,OHAROCGP,OHAROCOH,OHAROCCI,OHAROCDE,OHARNF,OHAROTH,OHAPOS
0,109263.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
1,109264.0,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
2,109265.0,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
3,109266.0,1.0,1.0,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
4,109269.0,1.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0


We don't need all the data, in order to make the data look clearer, we pick out the ones we need according to the definition of the variables. Based on the code book of OHXREF, we will choose:  
* SEQN (Respondent sequence number);  
* OHAROCDT(Decayed teeth)：  
1-Yes,  
2-NO;  
For futher analysis, we will delete null data.

In [122]:
selected_columns = ['SEQN', 'OHAROCDT']

# select columns of demo and rename
ohxref_selected = ohxref[selected_columns].copy()
ohxref_selected.columns = ['Respondent sequence number','Decayed teeth']

# Replace values in specified columns
replace_dict = {
    1: 'Yes',
    2: 'No',
}
ohxref_selected.replace(replace_dict, inplace=True)
# DROP null
ohxref_selected = ohxref_selected.dropna(subset=['Respondent sequence number','Decayed teeth'])

print(ohxref_selected.shape)
#Print
ohxref_selected.head()

(3256, 2)


,Respondent sequence number,Decayed teeth
6,109271.0,Yes
12,109279.0,Yes
23,109292.0,Yes
24,109293.0,Yes
36,109307.0,Yes


From the above data we can see that after screening the variables to be included in the analysis and removing the NaN values, for Demographic data it was reduced from 15560 to 13359, Body Measures data was reduced from 14300 to 13137, Blood Pressure data was reduced from 10195 to 1847, Dietary Supplement data was reduced from 15560 to 1633, and Oral Health data was reduced from 13772 to 3256, so the available analyzed variables for Dietary Supplement and Oral Health had more NaN values.  
Due to the existence of many NaN values, when different data sets are merged, we need to consider choosing the appropriate variables in order to ensure the amount of data.

In [123]:
# Merge DataFrames using 'Respondent sequence number' as the key
merged_df = pd.merge(demo_selected, bmx_selected, on='Respondent sequence number', how='outer')
merged_df = pd.merge(merged_df, bpq_selected, on='Respondent sequence number', how='outer')
merged_df = pd.merge(merged_df, dsqtot_selected, on='Respondent sequence number', how='outer')
merged_df = pd.merge(merged_df, ohxref_selected, on='Respondent sequence number', how='outer')

# Print the shape and the head of the merged DataFrame
print(merged_df.shape)
merged_df.head()

(15149, 15)


,Respondent sequence number,Gender,Age,Birth Country,Poverty,Weight,Height,Waist,BMI,Told had hypertension,Taking medicine for hypertension,Taking medicine for cholesterol,Total sugars,Vitamin C,Decayed teeth
0,109263.0,Male,2.0,USA,4.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,13.0,NaN
1,109264.0,Female,13.0,USA,0.83,42.2,154.7,63.8,17.6,NaN,NaN,NaN,NaN,NaN,NaN
2,109265.0,Male,2.0,USA,3.06,12.0,89.3,41.2,15.0,NaN,NaN,NaN,NaN,NaN,NaN
3,109266.0,Female,29.0,Others,5.00,97.1,160.2,117.9,37.8,NaN,NaN,NaN,NaN,NaN,NaN
4,109267.0,Female,21.0,Others,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Analysis and Visualization

Based on the above data organization, we will observe the relationship between the variables through Bokeh packages graphing.  
The relationships we want to look at include the gender distribution of the sample, is eating sugar related to tooth decay, does eating sugar make you fat, is Vitamin C really cardiovascularly protective, is there a relationship between BMI and age income, etc.?

In [124]:
# activate Bokeh output in Jupyter notebook
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [125]:
from bokeh.models import ColumnDataSource
df = ColumnDataSource(merged_df)

In [126]:
#data_table_plot
from bokeh.layouts import column
from bokeh.models import (ColumnDataSource, DataTable, HoverTool, IntEditor,
                          NumberEditor, NumberFormatter, SelectEditor,
                          StringEditor, StringFormatter, TableColumn)
from bokeh.plotting import figure, show

# Filter NaN. 
merged_df_filtered = merged_df.dropna()
# Create ColumnDataSource with the filtered DataFrame
source = ColumnDataSource(merged_df_filtered)

#manufacturers = sorted(mpg["manufacturer"].unique())
#models = sorted(mpg["model"].unique())
#transmissions = sorted(mpg["trans"].unique())
#drives = sorted(mpg["drv"].unique())
#classes = sorted(mpg["class"].unique())

columns = [
    TableColumn(field="Respondent sequence number", title="Respondent sequence number",
               editor=IntEditor()),
    TableColumn(field="Gender", title="Gender", editor=IntEditor()),
    TableColumn(field="Age", title="Age", editor=IntEditor()),
    TableColumn(field="Birth Country", title="Birth Country", editor=IntEditor()),
    TableColumn(field="Poverty", title="Poverty", editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0.0")),
    TableColumn(field="Weight", title="Weight",editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0.0")),
    TableColumn(field="Height", title="Height",editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0.0")),
    TableColumn(field="Waist", title="Waist",
                editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0.0")),
    TableColumn(field="BMI", title="BMI", editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0.0")),
    TableColumn(field="Told had hypertension", title="Told had hypertension", editor=IntEditor()),
    TableColumn(field="Taking medicine for hypertension", title="Taking medicine for hypertension", editor=IntEditor()),
    TableColumn(field="Taking medicine for cholesterol", title="Taking medicine for cholesterol", editor=IntEditor()),
    TableColumn(field="Total sugars", title="Total sugars", editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0.0")),
    TableColumn(field="Vitamin C", title="Vitamin C",editor=NumberEditor(step=0.1), formatter=NumberFormatter(format="0.0")),
    TableColumn(field="Decayed teeth", title="Decayed teeth", editor=IntEditor()),
]
data_table = DataTable(source=source, columns=columns, editable=True, width=800,
                       index_position=-1, index_header="row index", index_width=60)

p = figure(width=800, height=300, tools="pan,wheel_zoom,xbox_select,reset", active_drag="xbox_select")

#Gender = p.circle(x="index", y="Gender", fill_color="#396285", size=8, alpha=0.5, source=source)
Age = p.circle(x="index", y="Age", fill_color="#CE603D", size=8, alpha=0.5, source=source)

tooltips = [
    ("Respondent sequence number", "@{Respondent sequence number}"),
    ("Gender", "@{Gender}"),
    ("Birth Country", "@{Birth Country}"),
    #("Age", "@Age"),
    ("Weight", "@Weight"),
    ("Height", "@Height"),
    ("Waist", "@Waist"),
    ("BMI", "@BMI"),
    ("Told had hypertension", "@{Told had hypertension}"),
    ("Taking medicine for hypertension", "@{Taking medicine for hypertension}"),
    ("Taking medicine for cholesterol", "@{Taking medicine for cholesterol}"),
    ("Total sugars", "@{Total sugars}"),
    ("Vitamin C", "@{Vitamin C}"),
    ("Decayed teeth", "@{Decayed teeth}"),  
    
]
#Gender_hover_tool = HoverTool(renderers=[Gender], tooltips=[*tooltips, ("Gender", "@Gender")])
Age_hover_tool = HoverTool(renderers=[Age], tooltips=[*tooltips, ("Age", "@Age")])

p.add_tools(Age_hover_tool)

show(column(p, data_table))

From the above figure, we can see that only 21 samples contain all the dimensions Respondent sequence number, Gender, Age, Poverty, Weight, Height, Waist, BMI, Told had hypertension, Taking medicine for Hypertension, Taking medicine for cholesterol, Total sugars, Vitamin C, Decayed teeth, so we need to pay attention to the subsequent selection of variables.

In [127]:
import numpy as np
from bokeh.models import CustomJSTickFormatter, Label
from bokeh.palettes import DarkText, Vibrant3 as colors
from bokeh.plotting import figure, show

# Group by gender
sex_group = merged_df.groupby('Gender')

# Get age data for each gender
female_ages = sex_group.get_group('Female')['Age'].dropna()
male_ages = sex_group.get_group('Male')['Age'].dropna()

# Set the width of age bins
bin_width = 5
bins = np.arange(0, 72, bin_width)
m_hist, edges = np.histogram(male_ages, bins=bins)
f_hist, edges = np.histogram(female_ages, bins=bins)

# Create a figure
p = figure(title="Age Population Pyramid by Gender", height=400, width=600,
           x_range=(-max(m_hist.max(), f_hist.max()), max(m_hist.max(), f_hist.max())), x_axis_label="count")

# Plot horizontal bars for male and female populations
p.hbar(right=-m_hist, y=edges[1:], height=bin_width*0.8, color=colors[1], line_width=0)
p.hbar(right=f_hist, y=edges[1:], height=bin_width*0.8, color=colors[0], line_width=0)

# Add text to every other bar
for i, (count, age) in enumerate(zip(f_hist, edges[1:])):
    if i % 2 == 1:
        continue
    p.text(x=count, y=edges[1:][i], text=[f"{age-bin_width}-{age}yrs"],
           x_offset=5, y_offset=7, text_font_size="12px", text_color=DarkText[5])

# Customize x-axis and y-axis
p.xaxis.ticker = (-1000,-800, -600, -400, -200, 0, 200, 400, 600, 800, 1000)
p.xaxis.major_tick_out = 0
p.y_range.start = 3
p.ygrid.grid_line_color = None
p.yaxis.visible = False

# Format tick labels as absolute values for the two-sided plot
p.xaxis.formatter = CustomJSTickFormatter(code="return Math.abs(tick);")

# Add labels
p.add_layout(Label(x=-150, y=72, text="Men", text_color=colors[1], x_offset=5))
p.add_layout(Label(x=20, y=72, text="Women", text_color=colors[0], x_offset=5))

# Show the plot
show(p)


From the graph above we see that the total number of males in the sample of respondents is more than females, and is more pronounced in the over 60s and under 10s.  
It is not clear to us whether the slight difference in the gender share of respondents is due to differences in the willingness of men and women to participate, and it is not clear to us whether this has an impact on the unbiased nature of the data collected. Due to the large total sample size, we will not differentiate between genders separately in subsequent studies.  
In the following, we will investigate whether there is a correlation between sugar and tooth decay, which we will analyze by drawing the group bar for Total Sugars and Decayed Teeth.

In [128]:
# Filter the DataFrame to include only non-NaN values in 'Total sugars' and 'Decayed teeth'
df_filtered = merged_df.dropna(subset=['Birth Country','Total sugars', 'Decayed teeth'])
df_filtered = df_filtered[df_filtered['Decayed teeth'] == 'Yes']
df_filtered = df_filtered[['Respondent sequence number', 'Birth Country','Total sugars', 'Decayed teeth']]
#Print
print(df_filtered.shape)
df_filtered.head()

(227, 4)


,Respondent sequence number,Birth Country,Total sugars,Decayed teeth
64,109334.0,USA,1.75,Yes
150,109434.0,USA,3.00,Yes
169,109456.0,USA,3.00,Yes
210,109505.0,USA,1.00,Yes
211,109506.0,USA,1.00,Yes


In [129]:
import numpy as np
from sklearn.neighbors import KernelDensity

from bokeh.models import ColumnDataSource, Label, PrintfTickFormatter
from bokeh.palettes import Dark2_5 as colors
from bokeh.plotting import figure, show

# Sample data - Replace with your actual data
# merged_df = df_filtered

# Split the data by 'Birth Country'
usa_data = df_filtered[df_filtered['Birth Country'] == 'USA']['Total sugars'].values
others_data = df_filtered[df_filtered['Birth Country'] == 'Others']['Total sugars'].values

# Create x-axis coordinates
x = np.linspace(0, max(max(usa_data), max(others_data)), 1000)

# Create a Bokeh plot
p = figure(title="Kernel Density Estimates of Total Sugars by Birth Country", height=300, x_range=(0, max(x)),
           x_axis_label="Total Sugars", y_axis_label="Density")

# Use KernelDensity to estimate density
for data, label, color in [(usa_data, 'USA', 'red'), (others_data, 'Others', 'blue')]:
    kde = KernelDensity(kernel="gaussian", bandwidth=0.2).fit(data[:, np.newaxis])
    log_density = kde.score_samples(x[:, np.newaxis])
    y = np.exp(log_density)

    # Create a data source
    source = ColumnDataSource(data={'x': x, 'y': y})

    # Use varea() to plot the KDE curve
    p.varea(x='x', y1=0, y2='y', source=source, fill_color=color, fill_alpha=0.3, legend_label=label)

# Set the plot style
p.legend.title = "Birth Country"
p.xaxis.formatter = PrintfTickFormatter(format="%d")

# Hide axis lines and set y-axis ticks
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.minor_tick_line_color = None
p.xgrid.grid_line_color = None
p.yaxis.ticker = (0, 0.5, 1, 1.5)
p.y_range.start = 0

# Show the plot
show(p)

As we can see from the graph above, Among respondents with DECAYED teeth, U.S.-born respondents were more likely to get more sugar. Since the sample of decayed teeth was 10,516 out of 13,772 respondents for missing, there are no valid data, and our full sample was 228 with 1 non-decayed tooth, the samples of missing may have potentially huge implications. Therefore it is uncertain whether sugar consumption necessarily leads to tooth decay, for example whether the effect is attenuated if the respondent visits a dentist, whether tooth decay and household income are also correlated, and whether the respondent's other dietary habits also have an effect, therefore the relationship between sugar consumption and tooth decay needs to be analyzed by incorporating more factors.
We also want to know if there is a relationship between sugar and BMI, which can be seen through a scatter plot.

In [130]:
# Filter the DataFrame to include only non-NaN values in 'Total sugars' and 'Decayed teeth'
df_filtered2 = merged_df.dropna(subset=['Waist','Height','BMI', 'Total sugars'])
df_filtered2 = df_filtered2[['Respondent sequence number','Waist','Height', 'BMI', 'Total sugars']]
#Print
print(df_filtered2.shape)
df_filtered2.head()

(1345, 5)


,Respondent sequence number,Waist,Height,BMI,Total sugars
36,109304.0,60.8,137.7,16.3,1.07
42,109310.0,77.4,136.3,22.4,2.50
44,109312.0,95.7,166.5,23.9,3.00
64,109334.0,88.1,156.1,24.5,1.75
71,109341.0,57.8,125.4,15.8,1.50


In [131]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.transform import jitter
from bokeh.palettes import Category10_3

Body = ['Waist', 'Height', 'BMI']

source = ColumnDataSource(df_filtered2)

p = figure(width=800, height=300, title="Total Sugars VS Waist/Height/BMI")

for i, body_part in enumerate(Body):
    color = Category10_3[i]  # Choose different colors for body measures
    p.scatter(x='Total sugars', y=jitter(body_part, width=0.6, range=p.y_range), 
              source=source, alpha=0.3, legend_label=body_part, color=color)

p.legend.title = 'Body Measures'
p.legend.label_text_font_size = '10pt'
p.x_range.range_padding = 0
p.ygrid.grid_line_color = None

show(p)

From the graph above we can not see a significant positive relationship between sugar and Waist/Height/BMI. It's possible that fitness, other eating habits have an impact on BMI, sugar doesn't have as much of an impact.  
Some studies have suggested that Vitamin C is beneficial in mitigating cardiovascular disease, so we chose 'Told had hypertension', 'Taking medicine for hypertension', 'Taking medicine for cholesterol' to be compared with 'Vitamin C' respectively, and we did the following 3 group bar.

In [132]:
# Filter the DataFrame to include only non-NaN values in specified columns
df_filtered3 = merged_df.dropna(subset=['Vitamin C', 'Told had hypertension', 'Taking medicine for hypertension', 'Taking medicine for cholesterol'])
df_filtered3 = df_filtered3[['Respondent sequence number', 'Vitamin C', 'Told had hypertension', 'Taking medicine for hypertension', 'Taking medicine for cholesterol']]

# Print the shape and head of the filtered DataFrame
print(df_filtered3.shape)
#df_filtered3.head()

(102, 5)


In [133]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, FactorRange, LabelSet
from bokeh.transform import factor_cmap
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.palettes import MediumContrast3
output_notebook()

# Assuming df_filtered3 is your DataFrame
# Define the variables for plotting
y_columns = ['Told had hypertension', 'Taking medicine for hypertension', 'Taking medicine for cholesterol']
x_column = 'Vitamin C'
factors = ["Yes", "No"]

# Prepare the data for nested grouping
data = {'factors': [], 'values': []}
for y_column in y_columns:
    group_means = df_filtered3.groupby(y_column)[x_column].mean().reset_index()
    for factor in factors:
        data['factors'].append((y_column, factor))
        value = group_means.loc[group_means[y_column] == factor, x_column].iloc[0] if factor in group_means[y_column].values else 0
        data['values'].append(f"{value:.2f}") # Format the value to two decimal places

source = ColumnDataSource(data)

# Define the color mapping
color_map = factor_cmap('factors', palette=MediumContrast3, factors=factors, start=1, end=2)

# Create a figure with a nested x-axis
p = figure(x_range=FactorRange(*data['factors']), height=400,width=800,title=f"{x_column} VS {', '.join(y_columns)}",
           toolbar_location=None, tools="")

# Create the bar chart
p.vbar(x='factors', top='values', width=0.9, source=source, line_color="white", fill_color=color_map)
# Customize the plot
p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

# Adding labels to the bars
labels = LabelSet(x='factors', y='values', text='values', level='glyph', x_offset=-13.5, y_offset=5, source=source, text_font_size="8pt", text_color="black")
p.add_layout(labels)

# Show the plot
show(p)

Loading BokehJS ...

From the graph above we see that respondents who were not told they had hypertension had significantly higher VitaminC intake. I.e., with the dimension of Hypertension medication and Cholesterol medication, there is no way for us to confirm whether not taking the medication, is a matter of not having the disease or having the disease but not taking it. There was no significant difference in VC levels between respondents who took or did not take medication in VC and with hypertension medication. Whereas in VC and with cholesterol medication, respondents who did not take the medication significantly acquired more VC, but we could not confirm whether the patients were healthier or whether VC also had a beneficial effect on cholesterol. This still needs to be studied in more dimensions.  
Finally we would like to observe if there is a correlation between 'Gender', 'Age', 'Poverty', 'Weight', 'Height', 'Waist', 'BMI', which we will do by calculating the Pearson coefficient and drawing a heatmap.

In [134]:
# Filter the DataFrame to include only non-NaN values in specified columns
df_filtered4 = merged_df.dropna(subset=['Gender','Age','Poverty','Weight','Height','Waist','BMI'])
df_filtered4 = df_filtered4[['Respondent sequence number','Gender','Age','Poverty','Weight','Height','Waist','BMI']]

# Print the shape and head of the filtered DataFrame
print(df_filtered4.shape)
df_filtered4.head()

(10974, 8)


,Respondent sequence number,Gender,Age,Poverty,Weight,Height,Waist,BMI
1,109264.0,Female,13.0,0.83,42.2,154.7,63.8,17.6
2,109265.0,Male,2.0,3.06,12.0,89.3,41.2,15.0
3,109266.0,Female,29.0,5.00,97.1,160.2,117.9,37.8
7,109270.0,Female,11.0,1.88,75.3,156.0,91.4,30.9
9,109273.0,Male,36.0,0.83,74.3,184.2,86.8,21.9


In [135]:
import numpy as np
from bokeh.models import ColorBar, LinearColorMapper
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.transform import transform

output_notebook()

# Group by Gender
male_data = df_filtered4[df_filtered4['Gender'] == 'Male']
female_data = df_filtered4[df_filtered4['Gender'] == 'Female']
variables = ['Age', 'Poverty', 'Weight', 'Height', 'Waist', 'BMI']

def correlation_matrix(data, variables):
    data = data[variables].dropna()
    corr_matrix = data.corr()
    return corr_matrix

male_corr_matrix = correlation_matrix(male_data, variables)
female_corr_matrix = correlation_matrix(female_data, variables)

from bokeh.models import ColorBar, LinearColorMapper, BasicTicker
from bokeh.plotting import figure

def create_heatmap(matrix, title):
    size = len(matrix)
    xname = list(matrix.columns)
    yname = list(matrix.index)
    colors = matrix.values.flatten()

    data = {'xname': np.repeat(xname, size), 'yname': np.tile(yname, size), 'colors': colors}
    source = ColumnDataSource(data)

    mapper = LinearColorMapper(palette="Viridis256", low=-1, high=1)

    p = figure(title=title, x_range=xname, y_range=yname, x_axis_location="above",
               width=500, height=400, tools="hover", tooltips=[('variables', '@yname, @xname'), ('correlation', '@colors')])

    p.rect('xname', 'yname', width=1, height=1, source=source, line_color=None, fill_color=transform('colors', mapper))
    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "10pt"
    p.axis.major_label_standoff = 0

    color_bar = ColorBar(color_mapper=mapper, ticker=BasicTicker(desired_num_ticks=10), location=(0, 0))
    p.add_layout(color_bar, 'right')

    return p

male_heatmap = create_heatmap(male_corr_matrix, "Male Correlation Heatmap")
female_heatmap = create_heatmap(female_corr_matrix, "Female Correlation Heatmap")

show(male_heatmap)
show(female_heatmap)


Loading BokehJS ...

As can be seen from the above figure, there is little difference in the heatmap color between the male and female samples, so gender differences have little effect on the correlation of these variables.  
BMI has a correlation of 0.45 with Age, about 0.5 with Height, and more than 0.9 with Waist, and BMI has a relationship of 0.94 with Waist.The Poverty dimension, which represents family income, has less correlation with each of the dimensions of Age and Body measures. Therefore, a larger Waist can be considered to directly lead to a larger BMI, which is also in line with the laws of nature.The higher the Age, the greater the likelihood that the BMI will also increase.

### Conclusion

The analysis of the NHANES datasets has provided valuable insights into various aspects of the sampled population's health. Our examination of gender distribution, the relationship between sugar consumption and tooth decay, the impact of sugar on BMI, and the potential cardiovascular benefits of Vitamin C has shed light on important health considerations. The heatmap analysis further revealed correlations between age, body measures, and poverty, emphasizing the interconnectedness of these factors. While certain associations were observed, the complex nature of health-related interactions calls for further investigation and consideration of additional variables to draw conclusive insights. Overall, this study serves as a foundation for future research and underscores the importance of comprehensive analyses in understanding health dynamics in diverse populations.

### Reference
Centers for Disease Control and Prevention. (2017-2020). NHANES - National Health and Nutrition Examination Survey. Retrieved from https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&Cycle=2017-2020 
Rasmussen, C. B., Glisson, J. K., & Minor, D. S. (2012). Dietary Supplements and Hypertension: Potential Benefits and Precautions. The Journal of Clinical Hypertension, 14 May 2012. https://doi.org/10.1111/j.1751-7176.2012.00642.x  
Bokeh Development Team. (n.d.). Bokeh User Guide. Retrieved from https://docs.bokeh.org/en/latest/docs/user_guide.html